## Overview of the ETL Workflow

##### This notebook walks through an ETL process, which stands for Extract, Transform, and Load. You’ll see how to bring in different types of data, tidy them up, create new useful information, and save the cleaned results for further analysis.

In [ ]:
import numpy as np
import pandas as pd

### Data Collection (Extract)

##### First, we gather data from several sources including CSV files, JSON files, XML files, and databases. Learning to fetch data from various formats is crucial in real-world data projects.

In [ ]:
import os
for dirname, _, filenames in os.walk(r'N:\ETL\raw_files'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


#### Getting to Know Your Data

##### We inspect the first few rows and check for empty or missing values to better understand what problems the dataset might have. This helps us decide how to clean and prepare the data.

In [ ]:
df_projects = pd.read_csv('N:/ETL/raw_files/population_data.csv', dtype='str')

In [ ]:
df_projects.head()

In [ ]:
df_projects.isnull().sum()

In [ ]:
df_projects.shape

In [ ]:
df_population = pd.read_csv('N:/ETL/raw_files/population_data.csv')
df_population.head()



In [ ]:
df_population.isnull().sum()

In [ ]:
df_population.isnull().sum(axis=1)

In [ ]:
df_population = df_population.drop('Unnamed: 62', axis=1)

In [ ]:
df_population[df_population.isnull().any(axis=1)]

In [ ]:
def print_lines(n, file_name):
    f = open(file_name)
    for i in range(n):
        print(f.readline())
    f.close()

In [ ]:
print_lines(1, 'N:/ETL/raw_files/population_data.json')

In [ ]:
df_json = pd.read_json('N:/ETL/raw_files/population_data.json',orient='records')
df_json.head()

In [ ]:
import json

# read in the JSON file

with open('N:/ETL/raw_files/population_data.json') as f:
    json_data = json.load(f)

# read the first record in the JSON file
print(json_data[0])
print('\n')

# show that JSON data is essentially a dictionary
print(json_data[0]['Country Name'])
print(json_data[0]['Country Code'])

In [ ]:
print_lines(15, 'N:/ETL/raw_files/population_data.xml')

In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup

with open('N:/ETL/raw_files/population_data.xml') as fp:
    soup = BeautifulSoup(fp, "lxml")


In [ ]:
i = 0
# use the find_all method to get all record tags in the document
for record in soup.find_all('record'):
    # use the find_all method to get all fields in each record
    i += 1
    for record in record.find_all('field'):
        print(record['name'], ': ' , record.text)
    print()
    if i == 5:
        break

In [ ]:
import sqlite3

#connection to the database 
conn = sqlite3.connect('N:/ETL/raw_files/population_data.db')

#run a query
pd.read_sql('SELECT * FROM population_data', conn)

In [ ]:
pd.read_sql('SELECT "Country_Name", "Country_Code", "1960" FROM population_data', conn)

In [ ]:
import requests
import pandas as pd

url = 'http://api.worldbank.org/v2/countries/br;cn;us;de/indicators/SP.POP.TOTL/?format=json&per_page=1000'
r = requests.get(url)  # Make sure this line is executed first

r = requests.get(url)
r.json()


In [ ]:
url = 'http://api.worldbank.org/v2/country/CH/indicator/SP.POP.TOTL/?format=json&date=1995:2001'

# TODO: send the request
r = requests.get(url)
r.json()

### Combining datasets

#### Data Cleaning and Preparation

##### Cleaning involves fixing errors, removing unnecessary information, and dealing with missing or repeated values. Properly prepared data helps make sure our analysis is reliable and clear.

In [ ]:
f = open('N:/ETL/raw_files/rural_population_percent.csv')
for i in range(10):
    line = f.readline()
    print('line: ', i, line)
f.close()

In [ ]:
df_rural = pd.read_csv('N:/ETL/raw_files/rural_population_percent.csv',skiprows=4)
df_rural.head()

In [ ]:
f = open('N:/ETL/raw_files/electricity_access_percent.csv')
for i in range(10):
    line = f.readline()
    print('line: ', i, line)
f.close()

In [ ]:
df_electricity = pd.read_csv('N:/ETL/raw_files/electricity_access_percent.csv',skiprows=4)
df_electricity.head()

In [ ]:
df_rural.drop(['Unnamed: 62'],axis=1,inplace=True)

In [ ]:
df_electricity.drop(['Unnamed: 62'],axis=1,inplace=True)

In [ ]:
df_rural.head()

In [ ]:
df = pd.concat([df_rural, df_electricity])

In [ ]:
df.head()

#### Dealing with Missing Information

##### When data is incomplete, we fill in the blanks using methods like averaging available data or copying nearby values. This helps prevent mistakes during calculations or modeling.

In [ ]:
df_indicator = pd.read_csv('N:/ETL/raw_files/population_data.csv')
df_indicator.drop(['Unnamed: 62'], axis=1, inplace=True)

# read in the projects data set with all columns type string
df_projects = pd.read_csv('N:/ETL/raw_files/projects_data.csv', dtype=str)
df_projects.drop(['Unnamed: 56'], axis=1, inplace=True)

In [ ]:
df_indicator[['Country Name', 'Country Code']].drop_duplicates()

In [ ]:
df_projects['countryname'].unique()

In [ ]:
df_projects['Official Country Name'] = df_projects['countryname'].str.split(';').str.get(0)

In [ ]:
!pip install pycountry
from pycountry import countries

In [ ]:
countries.get(name='Spain')

In [ ]:
countries.lookup('Kingdom of Spain')

In [ ]:
from collections import defaultdict
country_not_found = [] # stores countries not found in the pycountry library
project_country_abbrev_dict = defaultdict(str) # set up an empty dictionary of string values

# iterate through the country names in df_projects. 
# Create a dictionary mapping the country name to the alpha_3 ISO code
for country in df_projects['Official Country Name'].drop_duplicates().sort_values():
    try: 
        # look up the country name in the pycountry library
        # store the country name as the dictionary key and the ISO-3 code as the value
        project_country_abbrev_dict[country] = countries.lookup(country).alpha_3
    except:
        # If the country name is not in the pycountry library, then print out the country name
        # And store the results in the country_not_found list
        print(country, ' not found')
        country_not_found.append(country)

In [ ]:
indicator_countries = df_indicator[['Country Name', 'Country Code']].drop_duplicates().sort_values(by='Country Name')

for country in country_not_found:
    if country in indicator_countries['Country Name'].tolist():
        print(country)

In [ ]:
country_not_found_mapping = {'Co-operative Republic of Guyana': 'GUY',
             'Commonwealth of Australia':'AUS',
             'Democratic Republic of Sao Tome and Prin':'STP',
             'Democratic Republic of the Congo':'COD',
             'Democratic Socialist Republic of Sri Lan':'LKA',
             'East Asia and Pacific':'EAS',
             'Europe and Central Asia': 'ECS',
             'Islamic  Republic of Afghanistan':'AFG',
             'Latin America':'LCN',
              'Caribbean':'LCN',
             'Macedonia':'MKD',
             'Middle East and North Africa':'MEA',
             'Oriental Republic of Uruguay':'URY',
             'Republic of Congo':'COG',
             "Republic of Cote d'Ivoire":'CIV',
             'Republic of Korea':'KOR',
             'Republic of Niger':'NER',
             'Republic of Kosovo':'XKX',
             'Republic of Rwanda':'RWA',
              'Republic of The Gambia':'GMB',
              'Republic of Togo':'TGO',
              'Republic of the Union of Myanmar':'MMR',
              'Republica Bolivariana de Venezuela':'VEN',
              'Sint Maarten':'SXM',
              "Socialist People's Libyan Arab Jamahiriy":'LBY',
              'Socialist Republic of Vietnam':'VNM',
              'Somali Democratic Republic':'SOM',
              'South Asia':'SAS',
              'St. Kitts and Nevis':'KNA',
              'St. Lucia':'LCA',
              'St. Vincent and the Grenadines':'VCT',
              'State of Eritrea':'ERI',
              'The Independent State of Papua New Guine':'PNG',
              'West Bank and Gaza':'PSE',
              'World':'WLD'}

In [ ]:
project_country_abbrev_dict.update(country_not_found_mapping)

In [ ]:
df_projects['Country Code'] = df_projects['Official Country Name'].apply(lambda x: project_country_abbrev_dict[x])

In [ ]:

df_projects[df_projects['Country Code'] == '']

#### data type

In [ ]:
df_projects = pd.read_csv('N:/ETL/raw_files/projects_data.csv', dtype=str)

In [ ]:
import pandas as pd

# read in the population data and drop the final column
df_indicator = pd.read_csv('N:/ETL/raw_files/population_data.csv')
df_indicator.drop(['Unnamed: 62'], axis=1, inplace=True)

# read in the projects data set with all columns type string
df_projects = pd.read_csv('N:/ETL/raw_files/projects_data.csv', dtype=str)
df_projects.drop(['Unnamed: 56'], axis=1, inplace=True)

In [ ]:
df_indicator.dtypes

In [ ]:
keepcol = ['Country Name']
for i in range(1960, 2018, 1):
    keepcol.append(str(i))

# In the df_nafta variable, store a data frame that only contains the rows for 
#      Canada, United States, and Mexico.
df_nafta = df_indicator[(df_indicator['Country Name'] == 'Canada') | 
             (df_indicator['Country Name'] == 'United States') | 
            (df_indicator['Country Name'] == 'Mexico')].iloc[:,]


# Calculate the sum of the values in each column in order to find the total population by year.
# You can use the keepcol variable if you want to control which columns get outputted
df_nafta.sum(axis=0)[keepcol]

In [ ]:
df_projects.dtypes

In [ ]:
df_projects[['totalamt', 'lendprojectcost']].head()

In [ ]:
df_projects['totalamt'].sum()

In [ ]:
df_projects['totalamt'] = pd.to_numeric(df_projects['totalamt'].str.replace(',',""))

#### Working with Time and Dates

##### We convert date-like text into special formats that let us easily extract parts like the year, month, and weekday, which helps in analyzing changes over time.

In [ ]:
parsed_date = pd.to_datetime('January 1st, 2017')
parsed_date

In [ ]:
parsed_date.month
parsed_date.year
parsed_date.second

In [ ]:
parsed_date = pd.to_datetime('5/3/2017 5:30')
parsed_date.month
parsed_date = pd.to_datetime('3/5/2017 5:30', format='%d/%m/%Y %H:%M')
parsed_date.month
parsed_date = pd.to_datetime('5/3/2017 5:30', format='%m/%d/%Y %H:%M')
parsed_date.month

In [ ]:
df_projects = pd.read_csv('N:/ETL/raw_files/projects_data.csv', dtype=str)
df_projects.drop(['Unnamed: 56'], axis=1, inplace=True)
df_projects.columns

In [ ]:
df_projects.head(15)[['boardapprovaldate', 'board_approval_month', 'closingdate']]

In [ ]:
df_projects['boardapprovaldate'] = pd.to_datetime(df_projects['boardapprovaldate'])
df_projects['closingdate'] = pd.to_datetime(df_projects['closingdate'])

In [ ]:
df_projects['boardapprovaldate'].dt.second
# Run this code cell to see the output
df_projects['boardapprovaldate'].dt.month
# Run this code to see the output
# weekday represents the day of the week from 0 (Monday) to 6 (Sunday).
df_projects['boardapprovaldate'].dt.weekday

In [ ]:
df_projects['approvalyear'] = df_projects['boardapprovaldate'].dt.year
df_projects['approvalday'] = df_projects['boardapprovaldate'].dt.day
df_projects['approvalweekday'] = df_projects['boardapprovaldate'].dt.weekday
df_projects['closingyear'] = df_projects['closingdate'].dt.year
df_projects['closingday'] = df_projects['closingdate'].dt.day
df_projects['closingweekday'] = df_projects['closingdate'].dt.weekday

In [ ]:
df = pd.read_csv('N:/ETL/raw_files/population_data.csv')

In [ ]:
from encodings.aliases import aliases

alias_values = set(aliases.values())

for encoding in set(aliases.values()):
    try:
        df=pd.read_csv("mystery.csv", encoding=encoding)
        print('successful', encoding)
    except:
        pass

#### imputing data

In [ ]:
df = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=4)
df.drop('Unnamed: 62', axis=1, inplace=True)

In [ ]:
df.head()
# Run this code cell to check how many null values are in the data set
df.isnull().sum()

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# put the data set into long form instead of wide
df_melt = pd.melt(df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='year', value_name='GDP')

# convert year to a date time
df_melt['year'] = pd.to_datetime(df_melt['year'])

def plot_results(column_name):
    # plot the results for Afghanistan, Albania, and Honduras
    fig, ax = plt.subplots(figsize=(8,6))

    df_melt[(df_melt['Country Name'] == 'Afghanistan') | 
            (df_melt['Country Name'] == 'Albania') | 
            (df_melt['Country Name'] == 'Honduras')].groupby('Country Name').plot('year', column_name, legend=True, ax=ax)
    ax.legend(labels=['Afghanistan', 'Albania', 'Honduras'])
    
plot_results('GDP')

##### excersixe 1

In [ ]:
df_melt['GDP_filled'] = df_melt.groupby('Country Name')['GDP'].transform(lambda x: x.fillna(x.mean()))
# Plot the results
plot_results('GDP_filled')

In [ ]:
df_melt['GDP_ffill'] = df_melt.sort_values('year').groupby('Country Name')['GDP'].fillna(method='ffill')
# plot the results
plot_results('GDP_ffill')

In [ ]:

df_melt['GDP_bfill'] = df_melt.sort_values('year').groupby('Country Name')['GDP'].fillna(method='bfill')
plot_results('GDP_bfill')

In [ ]:
# Run forward fill and backward fill on the GDP data
df_melt['GDP_ff_bf'] = df_melt.sort_values('year').groupby('Country Name')['GDP'].fillna(method='ffill').fillna(method='bfill')

# Check if any GDP values are null
df_melt['GDP_ff_bf'].isnull().sum()

### Duplicate data

##### Removing Repeated Entries Duplicate data can skew our results, so we search for and delete any repeated rows to get accurate insights.

In [ ]:
projects = pd.read_csv('N:/ETL/raw_files/projects_data.csv', dtype=str)
projects.drop('Unnamed: 56', axis=1, inplace=True)
projects['totalamt'] = pd.to_numeric(projects['totalamt'].str.replace(',', ''))
projects['countryname'] = projects['countryname'].str.split(';', expand=True)[0]
projects['boardapprovaldate'] = pd.to_datetime(projects['boardapprovaldate'])

# filter the data frame for projects over 1 billion dollars

# count the number of unique countries in the results

projects[projects['totalamt'] > 1000000000]['countryname'].nunique()

##### Converting Categories to Numbers (Dummy Variables)

##### Text categories such as sectors or regions are turned into numerical columns of 0s and 1s. This lets computers handle categorical data in calculations and machine learning models.

In [ ]:
projects = pd.read_csv('N:/ETL/raw_files/projects_data.csv', dtype=str)
projects.drop('Unnamed: 56', axis=1, inplace=True)
projects['totalamt'] = pd.to_numeric(projects['totalamt'].str.replace(',', ''))
projects['countryname'] = projects['countryname'].str.split(';', expand=True)[0]
projects['boardapprovaldate'] = pd.to_datetime(projects['boardapprovaldate'])

# keep the project name, lending, sector and theme data
sector = projects.copy()
sector = sector[['project_name', 'lendinginstr', 'sector1', 'sector2', 'sector3', 'sector4', 'sector5', 'sector',
          'mjsector1', 'mjsector2', 'mjsector3', 'mjsector4', 'mjsector5',
          'mjsector', 'theme1', 'theme2', 'theme3', 'theme4', 'theme5', 'theme ',
          'goal', 'financier', 'mjtheme1name', 'mjtheme2name', 'mjtheme3name',
          'mjtheme4name', 'mjtheme5name']]

In [ ]:
100 * sector.isnull().sum() / sector.shape[0]

In [ ]:
uniquesectors1 = sector['sector1'].sort_values().unique()
uniquesectors1
# run this code cell to see the number of unique values
print('Number of unique values in sector1:', len(uniquesectors1))

#### replace() methode

In [ ]:
sector['sector1'] = sector['sector1'].replace('!$!0', np.nan)

# TODO: In the sector1 variable, remove the last 10 or 11 characters from the sector1 variable.
# HINT: There is more than one way to do this including the replace method
# HINT: You can use a regex expression '!.+'
# That regex expression looks for a string with an exclamation
# point followed by one or more characters

sector['sector1'] = sector['sector1'].replace('!.+', '', regex=True)

# TODO: Remove the string '(Historic)' from the sector1 variable
# HINT: You can use the replace method
sector['sector1'] = sector['sector1'].replace('^(\(Historic\))', '', regex=True)

print('Number of unique sectors after cleaning:', len(list(sector['sector1'].unique())))
print('Percentage of null values after cleaning:', 100 * sector['sector1'].isnull().sum() / sector['sector1'].shape[0])

In [ ]:
dummies = pd.DataFrame(pd.get_dummies(sector['sector1']))

#  Filter the projects data for the totalamt, the year from boardapprovaldate, and the dummy variables
projects['year'] = projects['boardapprovaldate'].dt.year
df = projects[['totalamt','year']]
df_final = pd.concat([df, dummies], axis=1)

df_final.head()

#### Identifying and Visualizing Unusual Data Points (Outliers)

##### Some values are much higher or lower than the majority. We use statistical rules to find these outliers and visualize them using boxplots to understand their impact.

In [ ]:
# Read in the datasets and do basic wrangling
gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=4)
gdp.drop([col for col in ['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'] if col in gdp.columns], inplace=True, axis=1)

population = pd.read_csv('N:/ETL/raw_files/population_data.csv')
population.drop([col for col in ['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'] if col in population.columns], inplace=True, axis=1)

# Check actual column names
print("GDP columns:", gdp.columns)
print("Population columns:", population.columns)

# Use the correct column name for id_vars
# Find the correct column name for country in GDP and population datasets
country_col_gdp = next((col for col in gdp.columns if 'Country Name' in col), None)
country_col_population = next((col for col in population.columns if 'Country Name' in col), None)

if country_col_gdp is None or country_col_population is None:
    raise ValueError("Could not find 'Country Name' column in one of the datasets. Actual columns: GDP: {}, Population: {}".format(gdp.columns, population.columns))

# Reshape the data sets so that they are in long format
gdp_melt = gdp.melt(id_vars=[country_col_gdp],
                    var_name='year',
                    value_name='gdp')

# Use back fill and forward fill to fill in missing gdp values
gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby('Country Name')['gdp'].fillna(method='ffill').fillna(method='bfill')

population_melt = population.melt(id_vars=['Country Name'],
                                  var_name='year',
                                  value_name='population')

# Use back fill and forward fill to fill in missing population values
population_melt['population'] = population_melt.sort_values('year').groupby('Country Name')['population'].fillna(method='ffill').fillna(method='bfill')

# Merge the population and gdp data together into one data frame
df_country = gdp_melt.merge(population_melt, on=('Country Name', 'year'))

# Filter data for the year 2016
df_2016 = df_country[df_country['year'] == '2016']

# See what the data looks like
df_2016.head(10)

In [ ]:
import pandas as pd
import numpy as np

# Apply Tukey Rule to find outliers in population data for 2016
def tukey_outlier_analysis(data, column):
    """
    Apply Tukey Rule to identify outliers
    """
    print(f"Tukey Rule Outlier Analysis for {column.title()} - 2016")
    print("=" * 60)
    
    # Step 1: Find the first quartile (Q1 - 25th percentile)
    Q1 = data[column].quantile(0.25)
    print(f"First Quartile (Q1): {Q1:,.0f}")
    
    # Step 2: Find the third quartile (Q3 - 75th percentile)
    Q3 = data[column].quantile(0.75)
    print(f"Third Quartile (Q3): {Q3:,.0f}")
    
    # Step 3: Calculate the inter-quartile range (IQR)
    IQR = Q3 - Q1
    print(f"Inter-Quartile Range (IQR): {IQR:,.0f}")
    
    # Step 4 & 5: Calculate outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    print(f"\nOutlier Bounds:")
    print(f"Lower Bound (Q1 - 1.5 * IQR): {lower_bound:,.0f}")
    print(f"Upper Bound (Q3 + 1.5 * IQR): {upper_bound:,.0f}")
    
    # Identify outliers
    outliers_lower = data[data[column] < lower_bound]
    outliers_upper = data[data[column] > upper_bound]
    all_outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    print(f"\nOutlier Results:")
    print(f"Total number of outliers: {len(all_outliers)}")
    print(f"Lower outliers (< {lower_bound:,.0f}): {len(outliers_lower)}")
    print(f"Upper outliers (> {upper_bound:,.0f}): {len(outliers_upper)}")
    
    # Display outlier countries
    if len(all_outliers) > 0:
        print(f"\nCountries identified as outliers:")
        print("-" * 40)
        for idx, row in all_outliers.sort_values(column, ascending=False).iterrows():
            outlier_type = "Upper" if row[column] > upper_bound else "Lower"
            print(f"{row['Country Name']}: {row[column]:,.0f} ({outlier_type})")
    else:
        print("No outliers found.")
    
    # Summary statistics
    print(f"\nSummary Statistics for {column.title()}:")
    print(f"Minimum: {data[column].min():,.0f}")
    print(f"Maximum: {data[column].max():,.0f}")
    print(f"Mean: {data[column].mean():,.0f}")
    print(f"Median: {data[column].median():,.0f}")
    print(f"Standard Deviation: {data[column].std():,.0f}")
    
    return all_outliers

# Apply Tukey rule to population data
population_outliers = tukey_outlier_analysis(df_2016, 'population')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def plot_boxplot_with_outliers(data, column, title):
    """Create boxplot and label outliers with country names"""
    
    # Calculate quartiles and IQR
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    # Create the boxplot
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Create boxplot
    box_plot = ax.boxplot(data[column].dropna(), patch_artist=True)
    
    # Color the box
    box_plot['boxes'][0].set_facecolor('lightblue')
    box_plot['boxes'][0].set_alpha(0.7)
    
    # Add outlier points with labels
    for idx, row in outliers.iterrows():
        ax.plot(1, row[column], 'ro', markersize=8)  # Red dots for outliers
        ax.annotate(row['Country Name'], 
                   xy=(1, row[column]), 
                   xytext=(1.1, row[column]),
                   fontsize=9,
                   ha='left',
                   va='center',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
                   arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    
    ax.set_title(f'{title} - 2016', fontsize=14, fontweight='bold')
    ax.set_ylabel(column.title(), fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print outlier information
    print(f"\n{title} Outliers:")
    print("-" * 40)
    for idx, row in outliers.iterrows():
        print(f"{row['Country Name']}: {row[column]:,.0f}")

# Create boxplots with outlier labels
plot_boxplot_with_outliers(df_2016, 'population', 'Population Distribution')
plot_boxplot_with_outliers(df_2016, 'gdp', 'GDP Distribution')

In [ ]:
population_2016 = df_2016[['Country Name','population']]

# Calculate the first quartile of the population values for 2016
# HINT: you can use the pandas quantile method 
Q1 = population_2016['population'].quantile(0.25)

# Calculate the third quartile of the population values for 2016
Q3 = population_2016['population'].quantile(0.75)

# Calculate the interquartile range Q3 - Q1
IQR = Q3 - Q1

# Calculate the maximum value and minimum values according to the Tukey rule
# max_value is Q3 + 1.5 * IQR while min_value is Q1 - 1.5 * IQR
max_value = Q3 + 1.5 * IQR
min_value = Q1 - 1.5 * IQR

# filter the population_2016 data for population values that are greater than max_value or less than min_value
population_outliers = population_2016[(population_2016['population'] > max_value) | (population_2016['population'] < min_value)]
population_outliers

In [ ]:
non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

# remove non countries from the data
df_2016 = df_2016[~df_2016['Country Name'].isin(non_countries)]

In [ ]:
population_2016 = df_2016[['Country Name','population']]

# Calculate the first quartile of the population values
# HINT: you can use the pandas quantile method 
Q1 = population_2016['population'].quantile(0.25)

# Calculate the third quartile of the population values
Q3 = population_2016['population'].quantile(0.75)

# Calculate the interquartile range Q3 - Q1
IQR = Q3 - Q1

# Calculate the maximum value and minimum values according to the Tukey rule
# max_value is Q3 + 1.5 * IQR while min_value is Q1 - 1.5 * IQR
max_value = Q3 + 1.5 * IQR
min_value = Q1 - 1.5 * IQR

# filter the population_2016 data for population values that are greater than max_value or less than min_value
population_outliers = population_2016[(population_2016['population'] > max_value) | (population_2016['population'] < min_value)]
population_outliers

In [ ]:
# Filter the data for the year 2016 and put the results in the population_2016 variable. You only need
# to keep the Country Name and population columns
gdp_2016 = df_2016[['Country Name','gdp']]

# Calculate the first quartile of the population values
# HINT: you can use the pandas quantile method 
Q1 = gdp_2016['gdp'].quantile(0.25)

# Calculate the third quartile of the population values
Q3 = gdp_2016['gdp'].quantile(0.75)

# Calculate the interquartile range Q3 - Q1
IQR = Q3 - Q1

# Calculate the maximum value and minimum values according to the Tukey rule
# max_value is Q3 + 1.5 * IQR while min_value is Q1 - 1.5 * IQR
max_value = Q3 + 1.5 * IQR
min_value = Q1 - 1.5 * IQR

# filter the population_2016 data for population values that are greater than max_value or less than min_value
gdp_outliers = gdp_2016[(gdp_2016['gdp'] > max_value) | (gdp_2016['gdp'] < min_value)]
gdp_outliers

In [ ]:
# Find country names that are in both the population_outliers and the gdp_outliers 
# HINT: you can use the pandas intersection() method and python set() and list() methods

list(set(population_outliers['Country Name']).intersection(gdp_outliers['Country Name']))

In [ ]:

list(set(population_outliers['Country Name']) - set(gdp_outliers['Country Name']))

In [ ]:

list(set(gdp_outliers['Country Name']) - set(population_outliers['Country Name']))

In [ ]:
x = list(df_2016['population'])
y = list(df_2016['gdp'])
text = df_2016['Country Name']

fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(x, y)
plt.title('GDP vs Population')
plt.xlabel('population')
plt.ylabel('GDP')
for i, txt in enumerate(text):
    ax.annotate(txt, (x[i],y[i]))

In [ ]:
df_no_large = (df_2016['Country Name'] != 'United States') & (df_2016['Country Name'] != 'India') & (df_2016['Country Name'] != 'China')
x = list(df_2016[df_no_large]['population'])
y = list(df_2016[df_no_large]['gdp'])
text = df_2016[df_no_large]['Country Name']

fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(x, y)
plt.title('GDP vs Population')
plt.xlabel('population')
plt.ylabel('GDP')
for i, txt in enumerate(text):
    ax.annotate(txt, (x[i],y[i]))

In [ ]:
%pip install scikit-learn
from sklearn.linear_model import LinearRegression

# Check if df_2016 is empty before fitting the model
if df_2016.shape[0] > 0:
	model = LinearRegression()
	model.fit(df_2016['population'].values.reshape(-1, 1), df_2016['gdp'].values.reshape(-1, 1))

	# plot the data along with predictions from the linear regression model
	inputs = np.linspace(1, 2000000000, num=50)
	predictions = model.predict(inputs.reshape(-1,1))

	df_2016.plot('population', 'gdp', kind='scatter')
	plt.plot(inputs, predictions)
else:
	print("df_2016 is empty. Cannot fit LinearRegression model.")

In [ ]:
df_2016[df_2016['Country Name'] != 'United States'].plot('population', 'gdp', kind='scatter')
# plt.plot(inputs, predictions)
model.fit(df_2016[df_2016['Country Name'] != 'United States']['population'].values.reshape(-1, 1), 
          df_2016[df_2016['Country Name'] != 'United States']['gdp'].values.reshape(-1, 1))
inputs = np.linspace(1, 2000000000, num=50)
predictions = model.predict(inputs.reshape(-1,1))
plt.plot(inputs, predictions)

#### Eliminating Outliers

##### Removing extreme values helps us see the true patterns and trends in data without distortion.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
# Try reading with skiprows=4, but print columns to verify
gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=4)
print("GDP columns:", gdp.columns)
population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=4)
print("Population columns:", population.columns)

# If 'Country Name' is missing, try reading without skiprows or with skiprows=3
if 'Country Name' not in gdp.columns:
    gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=3)
    print("GDP columns after skiprows=3:", gdp.columns)
if 'Country Name' not in population.columns:
    population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=3)
    print("Population columns after skiprows=3:", population.columns)

# If still missing, try reading without skiprows
if 'Country Name' not in gdp.columns:
    gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv')
    print("GDP columns after skiprows=0:", gdp.columns)
if 'Country Name' not in population.columns:
    population = pd.read_csv('N:/ETL/raw_files/population_data.csv')
    print("Population columns after skiprows=0:", population.columns)

gdp.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')
population.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')

# Ensure 'Country Name' exists before melting
if 'Country Name' in gdp.columns:
    gdp_melt = gdp.melt(id_vars=['Country Name'], 
                        var_name='year', 
                        value_name='gdp')
    # Use back fill and forward fill to fill in missing gdp values
    gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby('Country Name')['gdp'].fillna(method='ffill').fillna(method='bfill')
else:
    raise KeyError("Column 'Country Name' not found in GDP data.")

if 'Country Name' in population.columns:
    population_melt = population.melt(id_vars=['Country Name'], 
                                      var_name='year', 
                                      value_name='population')
    # Use back fill and forward fill to fill in missing population values
    population_melt['population'] = population_melt.sort_values('year').groupby('Country Name')['population'].fillna(method='ffill').fillna(method='bfill')
else:
    raise KeyError("Column 'Country Name' not found in Population data.")

# merge the population and gdp data together into one data frame
df_country = gdp_melt.merge(population_melt, on=('Country Name', 'year'))

# filter data for the year 2016
df_2016 = df_country[df_country['year'] == '2016']

# filter out values that are not countries
non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

# remove non countries from the data
df_2016 = df_2016[~df_2016['Country Name'].isin(non_countries)]


# plot the data
x = list(df_2016['population'])
y = list(df_2016['gdp'])
text = df_2016['Country Name']

fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(x, y)
plt.title('GDP vs Population')
plt.xlabel('GDP')
plt.ylabel('Population')
for i, txt in enumerate(text):
    ax.annotate(txt, (x[i],y[i]))

In [ ]:
def tukey_rule(data_frame, column_name):
    data = data_frame[column_name]
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)

    IQR = Q3 - Q1

    max_value = Q3 + 1.5 * IQR
    min_value = Q1 - 1.5 * IQR
    
    return data_frame[(data_frame[column_name] < max_value) & (data_frame[column_name] > min_value)]

In [ ]:
df_outlier_removed = df_2016.copy()

for column in ['population','gdp']:
    df_outlier_removed = tukey_rule(df_outlier_removed, column)

In [ ]:
# plot the data
x = list(df_outlier_removed['population'])
y = list(df_outlier_removed['gdp'])
text = df_outlier_removed['Country Name']

fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(x, y)
plt.title('GDP vs Population')
plt.xlabel('GDP')
plt.ylabel('Population')
for i, txt in enumerate(text):
    ax.annotate(txt, (x[i],y[i]))

#### Bringing Numbers to a Common Scale (Scaling)

##### When data involves very large or small numbers, scaling helps by converting them to a comparable range, enhancing analysis and model performance.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# Read GDP data and check columns
gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=4)
if 'Country Name' not in gdp.columns:
    gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=3)
    if 'Country Name' not in gdp.columns:
        gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv')
print("GDP columns:", gdp.columns)
gdp.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')

# Read population data and check columns
population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=4)
if 'Country Name' not in population.columns:
    population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=3)
    if 'Country Name' not in population.columns:
        population = pd.read_csv('N:/ETL/raw_files/population_data.csv')
print("Population columns:", population.columns)
population.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')

# Reshape the data sets so that they are in long format
if 'Country Name' in gdp.columns:
    gdp_melt = gdp.melt(id_vars=['Country Name'], 
                        var_name='year', 
                        value_name='gdp')
    # Use back fill and forward fill to fill in missing gdp values
    gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby('Country Name')['gdp'].fillna(method='ffill').fillna(method='bfill')
else:
    raise KeyError("Column 'Country Name' not found in GDP data.")

if 'Country Name' in population.columns:
    population_melt = population.melt(id_vars=['Country Name'], 
                                      var_name='year', 
                                      value_name='population')
    # Use back fill and forward fill to fill in missing population values
    population_melt['population'] = population_melt.sort_values('year').groupby('Country Name')['population'].fillna(method='ffill').fillna(method='bfill')
else:
    raise KeyError("Column 'Country Name' not found in Population data.")

# merge the population and gdp data together into one data frame
df_country = gdp_melt.merge(population_melt, on=('Country Name', 'year'))

# filter data for the year 2016
df_2016 = df_country[df_country['year'] == '2016']

# filter out values that are not countries
non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area', 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

# remove non countries from the data
df_2016 = df_2016[~df_2016['Country Name'].isin(non_countries)]

# show the first ten rows
print('first ten rows of data')
df_2016.head(10)

#### Normalize the Data

In [ ]:
def x_min_max(data):
    minimum = min(data)
    maximum = max(data)
    return minimum, maximum

x_min_max(df_2016['gdp'])

In [ ]:
def normalize(x, x_min, x_max):
    # Complete this function
    # The input is a single value 
    # The output is the normalized value
    return (x - x_min) / (x_max - x_min)

In [ ]:
class Normalizer():
    # Complete the normalizer class
    # The normalizer class receives a dataframe as its only input for initialization
    # For example, the data frame might contain gdp and population data in two separate columns
    # Follow the TODOs in each section
    
    def __init__(self, dataframe):
        
        # complete the init function. 
        # Assume the dataframe has an unknown number of columns like [['gdp', 'population']] 
        # iterate through each column calculating the min and max for each column
        # append the results to the params attribute list
        
        # For example, take the gdp column and calculate the minimum and maximum
        # Put these results in a list [minimum, maximum]
        # Append the list to the params variable
        # Then take the population column and do the same
        
        # HINT: You can put your x_min_max() function as part of this class and use it
        
        self.params = []

        for column in dataframe.columns:
            self.params.append(x_min_max(dataframe[column]))
            
    def x_min_max(data):
        # complete the x_min_max method
        # HINT: You can use the same function defined earlier in the exercise
        minimum = min(data)
        maximum = max(data)
        return minimum, maximum

    def normalize_data(self, x):
        # complete the normalize_data method
        # The function receives a data point as an input and then outputs the normalized version
        # For example, if an input data point of [gdp, population] were used. Then the output would
        # be the normalized version of the [gdp, population] data point
        # Put the results in the normalized variable defined below
        
        # Assume that the columns in the dataframe used to initialize an object are in the same
        # order as this data point x
        
        # HINT: You cannot use the normalize_data function defined earlier in the exercise.
        # You'll need to iterate through the individual values in the x variable        
        # Use the params attribute where the min and max values are stored 
        normalized = []
        for i, value in enumerate(x):
            x_max = self.params[i][1]
            x_min = self.params[i][0]
            normalized.append((x[i] - x_min) / (x_max - x_min))
        return normalized

In [ ]:
gdp_normalizer = Normalizer(df_2016[['gdp', 'population']])
gdp_normalizer.params
gdp_normalizer.normalize_data([13424475000000.0, 1300000000])

#### Creating New Insights with Feature Engineering

##### We combine or transform existing data into new variables, like calculating GDP per person, to get deeper understanding and better analysis results.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# read in the projects data set and do basic wrangling 
gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=4)
if 'Country Name' not in gdp.columns:
    gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=3)
    if 'Country Name' not in gdp.columns:
        gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv')
gdp.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')

population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=4)
if 'Country Name' not in population.columns:
    population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=3)
    if 'Country Name' not in population.columns:
        population = pd.read_csv('N:/ETL/raw_files/population_data.csv')
population.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')


# Reshape the data sets so that they are in long format
gdp_melt = gdp.melt(id_vars=['Country Name'], 
                    var_name='year', 
                    value_name='gdp')

# Use back fill and forward fill to fill in missing gdp values
gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby('Country Name')['gdp'].fillna(method='ffill').fillna(method='bfill')

population_melt = population.melt(id_vars=['Country Name'], 
                                  var_name='year', 
                                  value_name='population')

# Use back fill and forward fill to fill in missing population values
population_melt['population'] = population_melt.sort_values('year').groupby('Country Name')['population'].fillna(method='ffill').fillna(method='bfill')

# merge the population and gdp data together into one data frame
df_country = gdp_melt.merge(population_melt, on=('Country Name', 'year'))

# filter data for the year 2016
df_2016 = df_country[df_country['year'] == '2016']

# filter out values that are not countries
non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
  'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

# remove non countries from the data
df_2016 = df_2016[~df_2016['Country Name'].isin(non_countries)]
df_2016.reset_index(inplace=True, drop=True)

In [ ]:
df_2016['gdppercapita'] = df_2016['gdp'] / df_2016['population']

#### Saving the Final Data (Load)

##### After cleaning and combining, we store the prepared data in files and databases so it can be used easily for reports, sharing, or further research.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# read in the projects data set and do basic wrangling 
gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=4)
population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=4)

# Check and adjust columns for GDP
if 'Country Name' not in gdp.columns:
    gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv', skiprows=3)
    if 'Country Name' not in gdp.columns:
        gdp = pd.read_csv('N:/ETL/raw_files/gdp_data.csv')
if 'Country Code' not in gdp.columns:
    gdp['Country Code'] = None  # Add a placeholder if missing

gdp.drop(['Unnamed: 62', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')

# Check and adjust columns for Population
if 'Country Name' not in population.columns:
    population = pd.read_csv('N:/ETL/raw_files/population_data.csv', skiprows=3)
    if 'Country Name' not in population.columns:
        population = pd.read_csv('N:/ETL/raw_files/population_data.csv')
if 'Country Code' not in population.columns:
    population['Country Code'] = None  # Add a placeholder if missing

population.drop(['Unnamed: 62', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1, errors='ignore')

# Reshape the data sets so that they are in long format
gdp_melt = gdp.melt(id_vars=['Country Name', 'Country Code'], 
                    var_name='year', 
                    value_name='gdp')

# Use back fill and forward fill to fill in missing gdp values
gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby(['Country Name', 'Country Code'])['gdp'].fillna(method='ffill').fillna(method='bfill')

population_melt = population.melt(id_vars=['Country Name', 'Country Code'], 
                                  var_name='year', 
                                  value_name='population')

# Use back fill and forward fill to fill in missing population values
population_melt['population'] = population_melt.sort_values('year').groupby(['Country Name', 'Country Code'])['population'].fillna(method='ffill').fillna(method='bfill')

# merge the population and gdp data together into one data frame
df_indicator = gdp_melt.merge(population_melt, on=('Country Name', 'Country Code', 'year'))

# filter out values that are not countries
non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
  'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

# remove non countries from the data
df_indicator  = df_indicator[~df_indicator['Country Name'].isin(non_countries)]
df_indicator.reset_index(inplace=True, drop=True)

df_indicator.columns = ['countryname', 'countrycode', 'year', 'gdp', 'population']

# output the first few rows of the data frame
df_indicator.head()

In [ ]:
!pip install pycountry
from pycountry import countries

# read in the projects data set with all columns type string
df_projects = pd.read_csv('N:/ETL/raw_files/projects_data.csv', dtype=str)
df_projects.drop(['Unnamed: 56'], axis=1, inplace=True)

df_projects['countryname'] = df_projects['countryname'].str.split(';').str.get(0)

# set up the libraries and variables
from collections import defaultdict
country_not_found = [] # stores countries not found in the pycountry library
project_country_abbrev_dict = defaultdict(str) # set up an empty dictionary of string values

# iterate through the country names in df_projects. 
# Create a dictionary mapping the country name to the alpha_3 ISO code
for country in df_projects['countryname'].drop_duplicates().sort_values():
    try: 
        # look up the country name in the pycountry library
        # store the country name as the dictionary key and the ISO-3 code as the value
        project_country_abbrev_dict[country] = countries.lookup(country).alpha_3
    except:
        # If the country name is not in the pycountry library, then print out the country name
        # And store the results in the country_not_found list
        country_not_found.append(country)
        
# run this code cell to load the dictionary

country_not_found_mapping = {'Co-operative Republic of Guyana': 'GUY',
             'Commonwealth of Australia':'AUS',
             'Democratic Republic of Sao Tome and Prin':'STP',
             'Democratic Republic of the Congo':'COD',
             'Democratic Socialist Republic of Sri Lan':'LKA',
             'East Asia and Pacific':'EAS',
             'Europe and Central Asia': 'ECS',
             'Islamic  Republic of Afghanistan':'AFG',
             'Latin America':'LCN',
              'Caribbean':'LCN',
             'Macedonia':'MKD',
             'Middle East and North Africa':'MEA',
             'Oriental Republic of Uruguay':'URY',
             'Republic of Congo':'COG',
             "Republic of Cote d'Ivoire":'CIV',
             'Republic of Korea':'KOR',
             'Republic of Niger':'NER',
             'Republic of Kosovo':'XKX',
             'Republic of Rwanda':'RWA',
              'Republic of The Gambia':'GMB',
              'Republic of Togo':'TGO',
              'Republic of the Union of Myanmar':'MMR',
              'Republica Bolivariana de Venezuela':'VEN',
              'Sint Maarten':'SXM',
              "Socialist People's Libyan Arab Jamahiriy":'LBY',
              'Socialist Republic of Vietnam':'VNM',
              'Somali Democratic Republic':'SOM',
              'South Asia':'SAS',
              'St. Kitts and Nevis':'KNA',
              'St. Lucia':'LCA',
              'St. Vincent and the Grenadines':'VCT',
              'State of Eritrea':'ERI',
              'The Independent State of Papua New Guine':'PNG',
              'West Bank and Gaza':'PSE',
              'World':'WLD'}

project_country_abbrev_dict.update(country_not_found_mapping)

df_projects['countrycode'] = df_projects['countryname'].apply(lambda x: project_country_abbrev_dict[x])

df_projects['boardapprovaldate'] = pd.to_datetime(df_projects['boardapprovaldate'])

df_projects['year'] = df_projects['boardapprovaldate'].dt.year.astype(str).str.slice(stop=4)

df_projects['totalamt'] = pd.to_numeric(df_projects['totalamt'].str.replace(',',""))

df_projects = df_projects[['id', 'countryname', 'countrycode', 'totalamt', 'year']]

df_projects.head()

In [ ]:
df_merged = df_projects.merge(df_indicator, how='left', on=['countrycode', 'year'])

In [ ]:
df_merged[(df_merged['year'] == '2017') & (df_merged['countryname_y'] == 'Jordan')]

In [ ]:
df_merged.to_json('countrydata.json', orient='records')

In [ ]:
df_merged.to_csv('countrydata.csv', index=False)

In [ ]:
import sqlite3

# connect to the database
# the database file will be worldbank.db
# note that sqlite3 will create this database file if it does not exist already
conn = sqlite3.connect('worldbank.db')

df_merged.to_sql('merged', con = conn, if_exists='replace', index=False)

In [ ]:
pd.read_sql('SELECT * FROM merged WHERE year = "2017" AND countrycode = "BRA"', con = conn).head()

In [ ]:
import sqlite3

# connect to the database
# the database file will be worldbank.db
# note that sqlite3 will create this database file if it does not exist already
conn = sqlite3.connect('worldbank.db')

df_indicator.to_sql('indicator', con = conn, if_exists='replace', index=False)
df_projects.to_sql('projects', con = conn, if_exists='replace', index=False)

In [ ]:
pd.read_sql('SELECT * FROM projects LEFT JOIN indicator ON \
projects.countrycode = indicator.countrycode AND \
projects.year = indicator.year WHERE \
projects.year = "2017" AND projects.countrycode = "BRA"', con = conn).head()

In [ ]:
conn.commit()
conn.close()